In [ ]:
####### 
# This cell is meant to install packages if some are missing
#######

# !pip install tqdm scipy
# !pip install matplotlib
# !pip install opencv-contrib-python
# !pip install numpy
# !pip install --upgrade tdmclient

In [ ]:

import matplotlib.pyplot as plt
import time
import numpy as np
from tqdm import tqdm
from kalmanfilter import KalmanFilter
import cv2   # a clean up
import Motion_control as mc

import aruco_vision
from djikstra import djikstra_algo
from calibration import calibration_mm, cam_calibration

### Connect the thymio(the speed set to 0 is here in case of an error during calibration or during the code)
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = aw(client.wait_for_node())
aw(node.lock())

mc.motors(node, 0, 0)

# Calibration

In this section, we will run multiple functions, in order to calibrate our thymio to the current environement. This gives us more flexibility, and makes it easier to use different thymio in different environment.

### Calibration of the thymio 

In this first step, we will use the following image to calibrate the speed, its conversion ratio from the thymio's sensor to mm/s, and its variance for the Kalman filter.
<!-- ![localisation map](Pictures/Picture3.jpg) -->
<img src="Pictures/Picture1.jpg" width="900"/>

In this next function, we will ask the thymio to move forward, at a known speed. As soon as its ground sensor detect it is on the black line, it will start a timer. When it exit the line, the timer will stop, the thymio will stop aswell, and all the desired values will be computed. In this function, the line length is know and we need to be careful to properly align the thymio as to make it follow the line. We can also adjust the speed to make it go as straight as possible, and adjust the transition threshold if the ligth's intensity in the current room requires it.

In [ ]:

thymio_data = []
SPEED_X = 101
SPEED_Y = 100
Ts = 0.01
TRANSITION_THRESHOLD = 400
LINE_LENGTH = 300


q_nu, speed_conversion = calibration_mm(node, Ts, thymio_data, LINE_LENGTH, TRANSITION_THRESHOLD, client, SPEED_X, SPEED_Y)
calibrated = 1



### Calibration of the camera
In this next cell, we will retrieve the the grid from the camera, and do all the conversion requiered for the rest of the project. This is done here, because we need to calibrate the image aswell, according to the position of the webcam. This function will make a conversion ratio between the image size and the known size of our grid.

In [ ]:
vision, Q_cam, R_cam, Q_gnd, R_gnd, SPEED_X, SPEED_Y, TRANSITION_THRESHOLD, Ts, speed_conversion = cam_calibration(calibrated)

In [3]:
# Ici on pourrait éventuellement calculer la taille de chaque case, et faire une variance pour kalman
print(vision.grid)


NameError: name 'vision' is not defined

## Run code

Now that we have everything set up, we can run the code

In [ ]:
#######
# The following values have been set in the calibration part
# 
# Ts = 0.01
# Speed = 60
# TRANSITION_THRESHOLD = 400
# If a preview is needed, run the following command
#
# print(f" The sampling time is : {Ts} s \n The speed is : {SPEED} mm/s \n The transition threshold is : {TRANSITION_THRESHOLD} \n")¨
# 
#######
#  The following values are the results of the calibration part 
#
# q_nu 
# r_nu
# conversion
# 
# print(f"The conversion factor for the speed of the thymio in mm/s is : {conversion} ")
# print(f"The standard deviation from the speed state (q_nu) and speed measurement (r_nu) is : {std_speed} ")
#######


In [ ]:
#same values as in the exercise session
vision.update_coordinates()
KF = KalmanFilter(Ts, vision.thymio_real_pos, speed_conversion, Q_cam, Q_gnd, R_cam, R_gnd)  # we initialize the filter

shortest_path = djikstra_algo(vision.grid.T, vision.thymio_position, vision.goal_position)
print(shortest_path)
print(f" goal position is : {vision.goal_position}")
print(f" thymio's position is : {vision.thymio_position}")


In [ ]:
step_duration = 125 / (SPEED_X * speed_conversion)
print(step_duration)

In [ ]:
step_duration = 125 / (SPEED_X * speed_conversion)
turn_duration = 98 / (SPEED_X * speed_conversion)
r_p_gnd = 0.25 
q_p_gnd = 0.04
vision.update_coordinates()
# orientation = mc.correct_orientation(vision.thymio_orientation)
x = vision.thymio_position[0]
y = vision.thymio_position[1]

#  A TOUT METTRE DANS MOTION CONTROL
jump = False
jump_x,jump_y = 0,0
turn_speed = 0
kalmann_pos_x, kalmann_pos_y = 0,0
ajdust = 0
for dx,dy in np.transpose(shortest_path):
    if jump:
        x = dx      #actualize the coordinates of the robot
        y = dy      #actualize the coordinates of the robot
        if jump_x == dx and jump_y == dy:
            jump = False
        continue
    
    # vision.update_coordinates()  #camera_check()
    # angle = vision.thymio_orientation #angle of the robot
    # if angle>=0:
    #     adjust_angle = angle - orientation*90
    # else:
    #     adjust_angle = angle - orientation*90 +360
    # motors(node,-SPEED*np.sign(adjust_angle), SPEED*np.sign(adjust_angle))
    # time.sleep(5*abs(adjust_angle)//180)
    # motors(node,0, 0)
    if False :#local_nav_error()
        jump_x,jump_y = 0,0#local_nav_correct(x,y,shortest_path)
    else:
        turn = mc.get_turn(dx-x,dy-y,orientation)
        orientation = orientation + turn
        mc.robot_turn(np.sign(turn),SPEED_X,speed_conversion, node, client)
        if (((dx-x)!=0) | ((dy-y)!=0)):
            start_move = time.time()
            step_done = False
            while (step_done == False):
                current = time.time()
                if((current - start_move) > step_duration):
                    step_done = True 
                else:   
                    # vision.update_coordinates()
                    vision.goal_position = (3,3)       
                    kalman_pos= KF.filter(1, vision.thymio_real_pos, orientation, (SPEED_X, SPEED_Y), 0, 0, 
                    TRANSITION_THRESHOLD)
                    adjust_turn,adjust_speed = mc.kalman_adjust(dx,dy,kalman_pos[0][0],kalman_pos[1][0],orientation)
                    kalman_pos_x = int(SPEED_X - adjust_turn - adjust_speed) #je reutilise juste les variables
                    kalman_pos_y = int(SPEED_Y + adjust_turn - adjust_speed)
                    mc.motors(node,kalman_pos_x+1, kalman_pos_y)
                    aw(client.sleep(Ts))
            mc.motors(node,0, 0)
    x = dx      #actualize the coordinates of the robot
    y = dy      #actualize the coordinates of the robot


We can see that the path has been properly computed. For the path's compution we needed the start and end point. As the kalman initialization also requires the starting point, we have decided to also initialize the filter at this point of the code.

In [ ]:
mc.motors(node, 100, 100)

In [ ]:
vision.update_coordinates()
print(vision.thymio_position)
print(vision.thymio_real_pos, vision.thymio_deviation)
if vision.img_final is None:
    print('Open Error')
else:
    plt.figure()
    plt.title('Original image')
    plt.imshow(cv2.cvtColor(vision.img_final.astype('uint8'), cv2.COLOR_BGR2RGB))

changer pour du 
move
start = time.time()

vbla bla bla

current = time.time()
if current-start == temps pour la durée désirée

alors on passe à l'étape suivante





jouer avec la mesure de la vitesse sur la phase de calibration

